In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
# credpath ='C:/Users/jakes/OneDrive/Documents/Python/credentials.json'
# credentials = credentials.Credentials.from_authorized_user_file(credpath)
# client = bigquery.Client(credentials=credentials)
client = bigquery.Client()

zonequery = """
SELECT TT.dma_name, term, SUM(score) as TOTAL_SCORE, RANK() OVER (ORDER BY SUM(score) DESC) AS TERM_RANK
FROM `bigquery-public-data.google_trends.top_terms` TT
WHERE
    1=1
   AND refresh_date = (SELECT MAX(refresh_date) FROM `bigquery-public-data.google_trends.top_terms`)
GROUP BY 1,2
ORDER BY 1 ASC,4 ASC;
"""
df = pd.read_gbq(zonequery)
totalquery = """
SELECT term, SUM(score) as TOTAL_SCORE, RANK() OVER (ORDER BY SUM(score) DESC) AS TERM_RANK
FROM `bigquery-public-data.google_trends.top_terms` TT
WHERE
    1=1
   AND refresh_date = (SELECT MAX(refresh_date) FROM `bigquery-public-data.google_trends.top_terms`)
GROUP BY 1
ORDER BY 3 ASC;
"""
appenddf = pd.read_gbq(totalquery)
appenddf['dma_name'] = 'TOTAL'
df = pd.concat([df,appenddf])
df.to_csv('C:/Users/jakes/Documents/TopGoogleSearchTerms.csv')

                 dma_name                      term  TOTAL_SCORE  TERM_RANK
0   Abilene-Sweetwater TX                       PSG         2756        773
1   Abilene-Sweetwater TX                 Bad Bunny         2716        827
2   Abilene-Sweetwater TX             American Idol         2687        859
3   Abilene-Sweetwater TX              Dolly Parton         2629        926
4   Abilene-Sweetwater TX  Liverpool vs Real Madrid         2608        947
..                    ...                       ...          ...        ...
20                  TOTAL            Meyers Leonard       269343         21
21                  TOTAL                     Putin       249200         22
22                  TOTAL            Jim Bob Cooter       233404         23
23                  TOTAL         Jansen Panettiere       230620         24
24                  TOTAL              Alec Baldwin       177309         25

[5275 rows x 4 columns]
